In [1]:
# import all python modules
import numpy as np
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
import scipy
import h5py
import tables
import glob
import itertools
import tables
import tqdm

# fill in the name of your study
studyname = "fill in studyname"  # fill in the studyname

# path were the raw data is (.bim .bam .bed or .vcf)
rawpath = "/home/charlesdarwin/plink/"    # fill in the path to the plink or vcf files


basepath = os.path.dirname(os.getcwd()) + "/"
hasepath = basepath + "/hase/"
savepath = basepath + "/processed_data/"

np.save(savepath + "studyname.npy", studyname)

In [2]:
basepath

'/home/avanhilten/PycharmProjects/GenNet/'

## Run HASE

In [ ]:
# Hase is used to convert data to the h5 format for fast reading and loading.
# Navigate to directory where you want to install HASE and clone this repository: 
# git clone https://github.com/roshchupkin/hase.git
# More information for conversion can be found here: https://github.com/roshchupkin/hase/wiki/Converting-Data

print("1. Open a new terminal")
print("2. Navigate to the map with hase (i.e. cd /home/charlesdarwin/hase/)")
print("\n")
print("3. Run: python hase.py -mode converting -g "+ rawpath + " -o "+hasepath+"  -study_name " + studyname)
print("\n")
print("If the raw data is in vcf format:")
print("3. Run: python hase.py -mode converting -g "+ rawpath + " -o "+hasepath+"  -study_name " + studyname + " -vcf")

## Merging the .h5 files

In [ ]:
print("Starting with processing... studyname = ", studyname)
filepath_hase = hasepath  + '/genotype/{}_'+studyname+'.h5'
g=h5py.File(filepath_hase.format(1) ,'r')['genotype']
num_pat  = g.shape[1]
number_of_files = len(glob.glob(hasepath + "/genotype/*.h5"))
print('number of files ',number_of_files) 


f = tables.open_file(savepath + studyname + '_genotype.h5', mode='w')
atom = tables.Int8Col()
filter_zlib = tables.Filters(complib='zlib', complevel=1)
array_c = f.create_earray(f.root, 'data', atom, (0, num_pat), filters=filter_zlib)
f.close()

print("\n merge all files...")
f = tables.open_file(savepath + studyname + '_genotype.h5', mode='a')
for i in tqdm.tqdm(range(number_of_files)): 
    gen_tmp = h5py.File(filepath_hase.format(i), 'r')['genotype']
    f.root.data.append(np.array(np.round(gen_tmp[:,:]), dtype=np.int))
f.close()



t = tables.open_file(savepath + studyname + '_genotype.h5', mode='r')
print('merged shape =', t.root.data.shape)
num_SNPS = t.root.data.shape[0]

p = pd.read_hdf(hasepath + '/probes/' + studyname + ".h5")
print('probe shape =', p.shape)

print("\n Impute...")
f = tables.open_file(savepath + studyname + '_genotype_imputed.h5', mode='w')
atom = tables.Int8Col()
num_pat = t.root.data.shape[1]
filter_zlib = tables.Filters(complib='zlib', complevel=1)
array_c = f.create_earray(f.root, 'data', atom, (0, num_pat), filters = filter_zlib)
f.close()

stdSNPs = np.zeros(num_SNPS)
f = tables.open_file(savepath + studyname + '_genotype_imputed.h5', mode='a')
for i in tqdm.tqdm(range(t.root.data.shape[0])):
    d=t.root.data[i,:].astype("float32")
    m=np.where(d == 9)
    m[0]
    d[m] = np.nan
    d[m] = np.nanmean(d)
    d= d[np.newaxis, :]
    f.root.data.append(np.round(d).astype(np.int8))
    stdSNPs[i] = np.std(d)
f.close()
t.close()


np.save(savepath + studyname + '_std.npy', stdSNPs)
print("Continue to 2, shut down this notebook to free up memory")